In [1]:
!pip install ollama

In [2]:
import ollama

In [3]:
prompt = """<s>[SYSTEM_PROMPT]
A user will ask you to solve a task. You should first draft your thinking process (inner monologue) until you have derived the final answer. Afterwards, write a self-contained summary of your thoughts (i.e. your summary should be succinct but contain all the critical steps you needed to reach the conclusion). You should use Markdown to format your response.

Your thinking process must follow the template below:
<think>
Your thoughts or/and draft, like working through an exercise on scratch paper. Be as casual and as long as you want until you are confident to generate a correct answer.
</think>

Here, provide a concise summary that reflects your reasoning and presents a clear final answer to the user. Don't mention that this is a summary.
[/SYSTEM_PROMPT][INST]
Here is a flawed math proof. Can you debug it step-by-step?

"Assume x = y. Then x² = xy. Subtract y² from both sides: x² - y² = xy - y². Factor both sides: (x + y)(x - y) = y(x - y). Cancel (x - y): x + y = y. Since x = y, we get 2y = y, so 2 = 1."
[/INST]"""


In [4]:
import ollama

response = ollama.chat(
    model="magistral",
    messages=[
        {
            "role": "user",
            "content": prompt,
        },
    ],
)
print(response["message"]["content"])

<think>
Alright, I have this flawed math proof to debug. Let's break it down step by step.

First, the proof starts with "Assume x = y". Okay, that's our initial condition. Then it says "Then x² = xy". Well, if x equals y, then substituting y for x in xy gives x² = xy. That seems correct so far.

Next: "Subtract y² from both sides: x² - y² = xy - y²". That's a valid operation—subtracting the same term from both sides maintains equality. Still looks good.

Then it says to factor both sides: (x + y)(x - y) = y(x - y). Okay, let's verify:
- Left side: x² - y² is a difference of squares, so it factors into (x + y)(x - y). Correct.
- Right side: xy - y² can be factored as y(x - y). So far, so good.

Next step: "Cancel (x - y): x + y = y". Hmm, this is where things might go wrong. Canceling terms from both sides of an equation is only valid if the term we're canceling isn't zero because division by zero is undefined. Here, (x - y) is being canceled, but since x = y (from the initial assumpti

In [ ]:
!pip install requests


In [ ]:
import requests

def call_ollama(prompt, model="magistral"):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": model, "prompt": prompt},
        stream=False  # Set True if you want token streaming
    )
    if response.ok:
        return response.json()['response']
    else:
        raise Exception(f"Ollama error: {response.text}")


In [ ]:
import gradio as gr
import requests
import json

def build_prompt(flawed_logic):
    return f"""<s>[SYSTEM_PROMPT]
A user will ask you to solve a task. You should first draft your thinking process (inner monologue) until you have derived the final answer. Afterwards, write a self-contained summary of your thoughts.

Your thinking process must follow the template below:
<think>
Your thoughts or/and draft, like working through an exercise on scratch paper. Be as casual and detailed as needed until you're confident.
</think>

Do not mention that you're debugging — just present your thought process and conclusion naturally.
[/SYSTEM_PROMPT][INST]
Here is a flawed solution. Can you debug it and correct it step-by-step?

\"\"\"{flawed_logic}\"\"\"
[/INST]
"""

def call_ollama_stream(flawed_logic):
    prompt = build_prompt(flawed_logic)
    response_text = ""

    with requests.post(
        "http://localhost:11434/api/generate",
        json={"model": "magistral", "prompt": prompt, "stream": True},
        stream=True,
    ) as r:
        for line in r.iter_lines():
            if line:
                content = json.loads(line).get("response", "")
                response_text += content
    return response_text

with gr.Blocks(theme=gr.themes.Base()) as demo:
    gr.Markdown("## Chain-of-Logic Debugger (Magistral + Ollama)")
    gr.Markdown("Paste a flawed logical argument or math proof, and Magistral will debug it with step-by-step reasoning.")

    with gr.Row():
        input_box = gr.Textbox(lines=8, label="Flawed Logic / Proof")
        output_box = gr.Textbox(lines=15, label="Debugged Explanation")
    
    debug_button = gr.Button("Run Debugger")

    debug_button.click(fn=call_ollama_stream, inputs=input_box, outputs=output_box)

demo.launch(debug = True, share=True)
